# Preparation

In [4]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 45

## Simulation setup

### Simulation params setup

In [5]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [7]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [8]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 3.75

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 14s, sys: 5.02 s, total: 3min 19s
Wall time: 3min 17s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 3]",96.85,"[-999, 6]","[0, 0]","[0, 0]"
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]"
2,2,"[2, 1]",39.95,"[1, 1]","[1.850000000000001, 0]","[0, 0]"
3,3,"[0, 3]",104.4,"[0, 13]","[0.05, 0]","[0, 0]"
4,4,"[2, 3]",71.35,"[2, -999]","[2.6999999999999984, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[0, 1]",169.2,"[73, 55]","[33.80000000000022, 0]","[0, 0]"
76,76,"[2, 1]",224.9,"[74, 74]","[88.59999999999711, 0]","[0, 0]"
77,77,"[2, 1]",229.25,"[76, 76]","[92.3999999999969, 0]","[0, 0]"
78,78,"[0, 3]",199.85,"[75, 73]","[33.75000000000022, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 3]",96.85,"[-999, 6]","[0, 0]","[0, 0]",1
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]",1
2,2,"[2, 1]",39.95,"[1, 1]","[1.850000000000001, 0]","[0, 0]",1
3,3,"[0, 3]",104.4,"[0, 13]","[0.05, 0]","[0, 0]",1
4,4,"[2, 3]",71.35,"[2, -999]","[2.6999999999999984, 0]","[0, 0]",1
5,5,"[0, 3]",106.65,"[3, 3]","[0.8500000000000002, 0]","[0, 0]",1
6,6,"[2, 3]",75.0,"[4, 4]","[3.599999999999995, 0]","[0, 0]",1
7,7,"[0, 1]",78.3,"[5, 14]","[2.0500000000000007, 0]","[0, 0]",1
8,8,"[0, 3]",117.9,"[7, 18]","[7.549999999999981, 0]","[2.849999999999998, 0]",1
9,9,"[0, 3]",121.6,"[8, 8]","[7.299999999999982, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.read_csv("../2.3.5.1 Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.676448,1.410693
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],8.128641,1.112257
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],6.895588,0.803888
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],8.014843,1.069629
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.623259,0.912343


## Summary of the 1st round

In [14]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,0,0.05,1,96.85
1,1,3,1.85,1,36.45
2,2,3,2.70,1,39.95
3,3,0,0.85,1,104.40
4,4,2,3.60,1,71.35
5,5,0,2.10,1,106.65
6,6,2,1.75,1,75.00
7,7,1,7.55,1,78.30
8,8,0,7.30,1,117.90
9,9,0,3.20,1,121.60


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[-31.475920765571637, 16.651092223153956, 16.6...","[4.183912311444762e-22, 0.33333333333333337, 0...",[0],7.676448,1.410693
1,1,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333333333196047, 0.33333333333196047, 0.3...",[3],8.128641,1.112257
2,2,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333333315527147, 0.33333333315527147, 0.3...",[3],6.895588,0.803888
3,3,"[-8.126372308580098, 16.651092223153956, 16.65...","[5.783125586983999e-12, 0.33333333333140563, 0...",[0],8.014843,1.069629
4,4,"[16.651092223153956, 16.651092223153956, -5.50...","[0.3333333333067458, 0.3333333333067458, 7.976...",[2],7.623259,0.912343
5,5,"[-6.281016553327932, 16.651092223153956, 16.65...","[3.660921741871906e-11, 0.3333333333211303, 0....",[0],8.155592,0.961688
6,6,"[16.651092223153956, 16.651092223153956, -3.97...","[0.3333333332114803, 0.3333333332114803, 3.655...",[2],8.271292,0.840350
7,7,"[16.651092223153956, -10.742820532344442, 16.6...","[0.3333333333331925, 4.2252615522695486e-13, 0...",[1],6.544596,1.093831
8,8,"[-13.239908295154999, 16.651092223153956, 16.6...","[3.4784211382828956e-14, 0.3333333333333217, 0...",[0],7.673765,1.126440
9,9,"[-11.131986278580971, 16.651092223153956, 16.6...","[2.863129961400235e-13, 0.33333333333323784, 0...",[0],7.379926,1.105813


In [15]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [16]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, 
                                all_routes, 
                                vehicle_limit, 
                                df, 
                                vehicle_preferences,
                                vehicle_rate, 
                                vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 2h 36min 16s, sys: 4min 1s, total: 2h 40min 18s
Wall time: 2h 34min 47s
